In [1]:
import pandas as pd

In [4]:
df = pd.read_csv("synthetic_dataset.csv", encoding_errors="replace")
instr_df = pd.read_csv("scored datasets/instruction_scores_added.csv", encoding_errors="replace")["instr_score"]
hallu_df = pd.read_csv("scored datasets/hallucination_scores_added.csv", encoding_errors="replace")["hallu_score"]
assum_df = pd.read_csv("scored datasets/assumption_scores_added.csv", encoding_errors="replace")["assump_score"]
accu_df = pd.read_csv("scored datasets/accuracy_scores_added.csv", encoding_errors="replace")["acc_score"]
coher_df = pd.read_csv("scored datasets/coherence_scores_added.csv", encoding_errors="replace")["coher_score"]

In [5]:
final_df = pd.concat([df,instr_df,hallu_df,assum_df,accu_df,coher_df], axis=1)
final_df.head()

question  \
0  What did the Court of Justice not acknowledge?   
1  What did the Court of Justice not acknowledge?   
2  What did the Court of Justice not acknowledge?   
3  What did the Court of Justice not acknowledge?   
4  What did the Court of Justice not acknowledge?   

                                             context gold_reference  \
0  Fourth, national courts have a duty to interpr...            NaN   
1  Fourth, national courts have a duty to interpr...            NaN   
2  Fourth, national courts have a duty to interpr...            NaN   
3  Fourth, national courts have a duty to interpr...            NaN   
4  Fourth, national courts have a duty to interpr...            NaN   

                                         ai_response  instr_score  \
0                    The passage doesn’t state this.          1.0   
1  It states that what did the court of justice w...          1.0   
2  The decision was influenced by the Paris Conve...          1.0   
3                               No answer in passage          1.0   
4  Bullet 1) The Court accepted horizontal effect...          1.0   

   hallu_score  assump_score  acc_score  coher_score  
0     0.443529      0.604477        0.0     1.000000  
1     0.176042      0.281990        0.0     1.000000  
2     0.182145      0.278720        0.0     1.000000  
3     0.118184      0.268096        0.0     0.925000  
4     0.245052      0.384162        0.0     0.759396

In [6]:
final_df["final_score"] = (
    0.15*final_df["instr_score"] +
    0.25*final_df["hallu_score"] +
    0.25*final_df["assump_score"] +
    0.20*final_df["acc_score"] +
    0.15*final_df["coher_score"]
)
final_df.head()

question  \
0  What did the Court of Justice not acknowledge?   
1  What did the Court of Justice not acknowledge?   
2  What did the Court of Justice not acknowledge?   
3  What did the Court of Justice not acknowledge?   
4  What did the Court of Justice not acknowledge?   

                                             context gold_reference  \
0  Fourth, national courts have a duty to interpr...            NaN   
1  Fourth, national courts have a duty to interpr...            NaN   
2  Fourth, national courts have a duty to interpr...            NaN   
3  Fourth, national courts have a duty to interpr...            NaN   
4  Fourth, national courts have a duty to interpr...            NaN   

                                         ai_response  instr_score  \
0                    The passage doesn’t state this.          1.0   
1  It states that what did the court of justice w...          1.0   
2  The decision was influenced by the Paris Conve...          1.0   
3                               No answer in passage          1.0   
4  Bullet 1) The Court accepted horizontal effect...          1.0   

   hallu_score  assump_score  acc_score  coher_score  final_score  
0     0.443529      0.604477        0.0     1.000000     0.562002  
1     0.176042      0.281990        0.0     1.000000     0.414508  
2     0.182145      0.278720        0.0     1.000000     0.415216  
3     0.118184      0.268096        0.0     0.925000     0.385320  
4     0.245052      0.384162        0.0     0.759396     0.421213

In [8]:
final_df.to_csv("scored datasets/final_scores.csv")

Now we will write code so as to get a dictionary or json file which contains each question with their generated question id and all the responses ranked with their ranking scoes against them

In [ ]:
import pandas as pd
import numpy as np

def to_text(x) -> str:
    if x is None:
        return ""
    try:
        import pandas as _pd
        if _pd.isna(x):
            return ""
    except Exception:
        pass
    return str(x)

def build_question_dict(
    df: pd.DataFrame,
    question_col: str = "question",
    context_col: str = "context",
    gold_col: str = "gold_reference",
    resp_col: str = "ai_response",
    score_col: str = "final_score",
    top_k: int = 5,
    question_id_col: str | None = None,   
    include_subscores: bool = False,      
    subscores_cols: tuple = ("instr_score","hallu_score","assump_score","acc_score","coher_score")
) -> dict:

    df = df.copy()

    # 1) Determine question IDs
    if question_id_col and question_id_col in df.columns:
        df["_qid"] = df[question_id_col]
    else:
        unique_q = df[question_col].astype(str).str.strip().drop_duplicates().tolist()
        qid_map = {q: i+1 for i, q in enumerate(unique_q)}
        df["_qid"] = df[question_col].astype(str).str.strip().map(qid_map)

    # 2) Build dictionary
    qdict = {}
    for qid, group in df.groupby("_qid", sort=False):
        g = group.copy()
        g[score_col] = pd.to_numeric(g[score_col], errors="coerce").fillna(0.0)
        g = g.sort_values(score_col, ascending=False).head(top_k)

        responses = []
        for rank, row in enumerate(g.itertuples(index=False), 1):
            entry = {
                "text": getattr(row, resp_col),
                "final_score": round(float(getattr(row, score_col)), 4),
                "rank": rank
            }
            if include_subscores:
                for c in subscores_cols:
                    if c in g.columns:
                        entry[c] = float(getattr(row, c)) if not pd.isna(getattr(row, c)) else None
            responses.append(entry)

        # take the first row's question/context/gold as canonical
        first = group.iloc[0]
        qdict[int(qid)] = {
            "question": to_text(first[question_col]),
            "context": to_text(first[context_col]),
            "gold_reference": to_text(first[gold_col]),
            "responses": responses
        }

    # cleaning temp column
    if "_qid" in df.columns:
        del df["_qid"]

    return qdict

In [23]:
question_dict = build_question_dict(final_df, top_k=5)

In [24]:
question_dict

{1: {'question': 'What did the Court of Justice not acknowledge?',
  'context': 'Fourth, national courts have a duty to interpret domestic law "as far as possible in the light of the wording and purpose of the directive". Textbooks (though not the Court itself) often called this "indirect effect". In Marleasing SA v La Comercial SA the Court of Justice held that a Spanish Court had to interpret its general Civil Code provisions, on contracts lacking cause or defrauding creditors, to conform with the First Company Law Directive article 11, that required incorporations would only be nullified for a fixed list of reasons. The Court of Justice quickly acknowledged that the duty of interpretation cannot contradict plain words in a national statute. But, fifth, if a member state has failed to implement a Directive, a citizen may not be able to bring claims against other non-state parties, but can sue the member state itself for failure to implement the law. So, in Francovich v Italy, the Ita

In [25]:
question_dict[2]

{'question': 'What does  oxygen therapy promote the body to take up?',
 'context': "Uptake of O\n2 from the air is the essential purpose of respiration, so oxygen supplementation is used in medicine. Treatment not only increases oxygen levels in the patient's blood, but has the secondary effect of decreasing resistance to blood flow in many types of diseased lungs, easing work load on the heart. Oxygen therapy is used to treat emphysema, pneumonia, some heart disorders (congestive heart failure), some disorders that cause increased pulmonary artery pressure, and any disease that impairs the body's ability to take up and use gaseous oxygen.",
 'gold_reference': 'gaseous oxygen.',
 'responses': [{'text': 'gaseous oxygen.', 'final_score': 0.757, 'rank': 1},
  {'text': 'Oxygen gas', 'final_score': 0.6022, 'rank': 2},
  {'text': 'It states that what does oxygen therapy promote the was fully acknowledged, including details not mentioned.',
   'final_score': 0.3972,
   'rank': 3},
  {'text': 

In [ ]:
# counting length of response and penalising deviation
question_dict[169]

{'question': 'Reply in exactly 5 words: Where does the sun rise?',
 'context': 'The sun rises in the east.',
 'gold_reference': 'The sun rises in east.',
 'responses': [{'text': 'The sun rises in east.',
   'final_score': 0.9474,
   'rank': 1},
  {'text': 'The sun rises in east. This answer answer repeats words badly.',
   'final_score': 0.5066,
   'rank': 2},
  {'text': 'The sun rises in east. Additionally, it also hints at broader implications beyond this scope.',
   'final_score': 0.4714,
   'rank': 3},
  {'text': 'The decision was influenced by the Paris Convention Committee, which clarified the issue, though the passage doesn’t mention this body.',
   'final_score': 0.3577,
   'rank': 4},
  {'text': 'Not exactly ten words and also includes punctuation errors and extras.',
   'final_score': 0.2745,
   'rank': 5}]}

In [ ]:
# identifying bullet points
question_dict[188]

{'question': 'Explain the problem and solution in two bullet points.',
 'context': 'Plastic waste is polluting oceans and harming marine life. Alternatives like biodegradable packaging are emerging.',
 'gold_reference': '• Problem: Plastic waste pollutes oceans, harms marine life.\n• Solution: Use biodegradable packaging as an alternative.',
 'responses': [{'text': '• Problem: Plastic waste pollutes oceans, harms marine life.\n• Solution: Use biodegradable packaging as an alternative.',
   'final_score': 0.5811,
   'rank': 1},
  {'text': 'It first concludes there is no recognition, and then immediately claims recognition happened twice, which is inconsistent and repeats the same point twice twice.',
   'final_score': 0.3168,
   'rank': 2},
  {'text': 'It both denies and affirms the recognition in the same breath, contradicting itself.',
   'final_score': 0.3082,
   'rank': 3},
  {'text': 'It states that explain the problem and solution in was fully acknowledged, including details not m

In [ ]:
# case where no context is there
question_dict[207]

{'question': 'What is the capital of France?',
 'context': '',
 'gold_reference': 'Paris',
 'responses': [{'text': 'Paris', 'final_score': 0.7388, 'rank': 1},
  {'text': 'In essence, paris, though expressed differently.',
   'final_score': 0.5786,
   'rank': 2},
  {'text': 'Paris This answer answer repeats words badly.',
   'final_score': 0.575,
   'rank': 3},
  {'text': 'Paris Additionally, it also hints at broader implications beyond this scope.',
   'final_score': 0.5667,
   'rank': 4},
  {'text': 'Bullet 1) The Court accepted horizontal effect in 2015; Bullet 2) therefore it never did so, clearly.',
   'final_score': 0.5139,
   'rank': 5}]}